In [ ]:
%pip install jira

## Helpers

In [163]:
class JiraInfo:
    user_auth_token = ''
    jira_server_address = ''
    basic_auth = ('login', 'password')
    user_login = ''

    def __init__(self, user_auth_token, user_login, jira_server_address, basic_auth):
        self.user_auth_token = user_auth_token
        self.user_login = user_login
        self.jira_server_address = jira_server_address
        self.basic_auth = basic_auth

In [164]:
import json

class UserWorklogInfoEncoder(json.JSONEncoder):
    def default(self, o):
        return o.__dict__ 

class UserWorklogInfo:
    comment = ''
    created_date = None
    time_spent_seconds = 0

    def __init__(self, comment, created_date, time_spent_seconds):
        self.comment = comment
        self.created_date = created_date
        self.time_spent_seconds = time_spent_seconds

In [165]:
import json

class JiraIssueEncoder(json.JSONEncoder):
    def default(self, o):
        return o.__dict__ 

class JiraIssue:
    code = ''
    name = ''
    worklogs = []

    def __init__(self, code, name, worklogs):
        self.code = code
        self.name = name
        self.worklogs = worklogs

In [166]:
def get_user_worklogs(issue, user_login):
    worklogs = issue.raw['fields']['worklog']['worklogs']
    worklogs = filter(lambda x: x['author']['name'] == user_login, worklogs)
    worklogs = map(lambda x: UserWorklogInfo(
        comment = x['comment'],
        created_date = x['created'],
        time_spent_seconds = x['timeSpentSeconds']
    ), worklogs)
    worklogs = list(worklogs)

    return worklogs

In [167]:
def map_issues(issue, user_login):
    issue_code = issue.raw['key']
    issue_name = issue.raw['fields']['summary']
    worklogs = get_user_worklogs(issue, user_login)
    # json_worklogs = json.dumps(worklogs, cls = UserWorklogInfoEncoder)
    mapped = JiraIssue(
        code = issue_code,
        name = issue_name,
        worklogs = worklogs
    )

    return mapped

In [174]:
import json

class UserWorklogItemDtoEncoder(json.JSONEncoder):
    def default(self, o):
        return o.__dict__ 

class UserWorklogItemDto:
    task_code = ''
    task_name = ''
    start_period_date = None
    end_period_date = None
    time_spent_seconds = 0

    def __init__(self, task_code, task_name, start_period_date, end_period_date, time_spent_seconds):
        self.task_code = task_code
        self.task_name = task_name
        self.start_period_date = start_period_date
        self.end_period_date = end_period_date
        self.time_spent_seconds = time_spent_seconds

In [196]:
def get_worklogs_period(worklogs):
    if not worklogs:
        return (None, None)
    
    select_dates = lambda x: x.created_date
    start_period = min(worklogs, key=select_dates).created_date
    end_period = max(worklogs, key=select_dates).created_date

    return (start_period, end_period)

def get_time_spent(worklogs):
    times = map(lambda x: x.time_spent_seconds, worklogs)
    time_spent = sum(times)

    return time_spent

def map_user_worklog(user_worklog):
    start_period_date, end_period_date = get_worklogs_period(user_worklog.worklogs)
    time_spent_seconds = get_time_spent(user_worklog.worklogs)
    user_worklog_item = UserWorklogItemDto(
        task_code = user_worklog.code,
        task_name = user_worklog.name,
        start_period_date = start_period_date,
        end_period_date = end_period_date,
        time_spent_seconds = time_spent_seconds
    )

    return user_worklog_item

## Code

In [168]:
user_login = 'JIRA_LOGIN'
jira_info = JiraInfo(
    user_auth_token = '',
    jira_server_address = 'https://jira.com/',
    basic_auth = (user_login, 'JIRA_PASSWORD'),
    user_login = user_login
)

In [169]:
from jira import JIRA

auth_jira = JIRA(
    basic_auth = jira_info.basic_auth,
    options = {'server': jira_info.jira_server_address }
)

In [170]:
import json

startWorklogDate = '2022-08-15'
endWorklogDate = '2022-09-05'
jql = f'worklogAuthor in ("{jira_info.user_login}") and worklogDate >= {startWorklogDate} and worklogDate < {endWorklogDate}'
found_issues = auth_jira.search_issues(jql)

In [ ]:
map_issue = lambda issue: map_issues(issue, jira_info.user_login)
user_worklogs = map(map_issue, found_issues)
user_worklog_items = list(map(map_user_worklog, user_worklogs))
json_user_worklogs = json.dumps(user_worklog_items, cls = UserWorklogItemDtoEncoder)
print(json_user_worklogs)